In [2]:
"""
AC Power flow - 25 bus case V1 (25.04.11)
Readme_ac_powerflow.md 참고
"""

""""
A. 파라미터 및 데이터 입력
"""
case_name = 'ac_case25'
case_file_name = 'ac_case25.xlsx'

""" Call modules """
import os
import pandas as pd
import numpy as np
import pyomo.environ as pyo 
import math

""""Packages"""
from Packages.Create_Y_bus import create_Y_bus
from Packages.Create_set_n_params import create_set_and_params

""""Path"""
Main_path = os.getcwd()
inputdata_path = Main_path + '/InputData/'
pre_caldata_path = Main_path + '/PreCalData/' # Y행렬, 전압, 유효전력 PU 초기값 등 조류계산 수행 전 계산값들이 저장되는 경로

print('Main path:'+Main_path)
print('Input data path:'+inputdata_path)
print('Precalculation data path:'+pre_caldata_path)

Main path:c:\Users\Woong\Desktop\Workfolder\Pyomo\For_git\Power_flow\ac_case25
Input data path:c:\Users\Woong\Desktop\Workfolder\Pyomo\For_git\Power_flow\ac_case25/InputData/
Precalculation data path:c:\Users\Woong\Desktop\Workfolder\Pyomo\For_git\Power_flow\ac_case25/PreCalData/


In [3]:
""""Data from excel"""
power_system_data = pd.ExcelFile(inputdata_path + case_file_name)

#power_system_data.sheet_names 참고
Bus_data = pd.read_excel(power_system_data,'bus')
Branch_data = pd.read_excel(power_system_data,'branch')
Transformer_data = pd.read_excel(power_system_data,'transformer')
Gen_data = pd.read_excel(power_system_data,'generator')
Param_data = pd.read_excel(power_system_data,'param')

# 데이터 입력 결과
Bus_data.head(3)

,Bus,Type,Pload (MW),Qload (MVAR),area,Vm (pu),Va (degree),maxVm,minVm
0,1,PV,108.0,22.00,1,1,0,1.05,0.95
1,2,PV,97.0,20.00,1,1,0,1.05,0.95
2,3,PQ,207.0,42.55,1,1,0,1.05,0.95


In [4]:
Branch_data.head(3)

,From,To,R (pu),X (pu),B (pu)
0,1,2,0.0026,0.0139,0.4611
1,1,3,0.0546,0.2112,0.0572
2,1,5,0.0218,0.0845,0.0229


In [5]:

Transformer_data.head(3)

,From,To,Tap
0,3,24,1.03
1,9,11,1.03
2,9,12,1.03


In [6]:
Gen_data.head(3)

,Id,Bus,PG (MW),QG (MVAR),QMAX (MVAR),QMIN (MVAR),Voltage setpoint (pu),MBASE (MW),STATUS
0,1,1,10.0,0.0,10,0,1.035,100,1
1,2,1,10.0,0.0,10,0,1.035,100,1
2,3,1,76.0,0.0,30,-25,1.035,100,1


In [7]:
Param_data.head(3)

,sbase (MVA),100


In [ ]:
""""
B. 데이터 전처리 (최적화 수행에 맞는 데이터 만들기)
"""

""" 1. 집합 만들기(Set) """
create_set_and_params(np,pd,pre_caldata_path,Bus_data['Bus'])

""" 2. Y 행렬 생성 """
#[Bus_i, Bus_j, Bus_G, Bus_B]를 열로 갖는 Y_bus.csv 파일 '/PreCalData/'폴더에 생성하는 것 목표
create_Y_bus(np,pd,pre_caldata_path,Bus_data['Bus'],Branch_data,Transformer_data)

""" 3. 모선별 아는 값 입력(전압의 크기, 위상, 유효 및 무효전력) """


In [ ]:
""""
C. 최적화 수행
"""

""" Pyomo INPUTS """
solver = 'ipopt' # Select solver

""" Pyomo Config """
# Create the abstract model - 시그마로 표현할 수 있는 모델
model = pyo.AbstractModel()

## Loaded from csv in ../PreCalData/
# Define set
model.Buses = pyo.Set(dimen=1)

# Define parameters
model.Bus_G = pyo.Param(model.Buses,model.Buses,within=pyo.Any) # Network conductivity matrix
model.Bus_B = pyo.Param(model.Buses,model.Buses,within=pyo.Any) # Network susceptance matrix


""" 목적함수 및 제약조건 생성"""

' 목적함수 및 제약조건 생성'

In [ ]:
"""
D. 결과 출력
"""
